In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline
import nibabel as nib
import numpy.ma as ma
from numpy import savetxt
import os
from brainspace.datasets import load_group_fc, load_parcellation, load_conte69
import brainstat.mesh.data as mesh

#input variables and path defs
#workDir = '/data_/mica1/03_projects/hans/7T/func/'
workDir='/host/percy/local_raid/hans/amyg/func/'
outDir = str(workDir+'outputs/')
volDir = '/data_/mica3/BIDS_PNC/rawdata/'
regDir = '/data_/mica1/03_projects/hans/7T/registration/'
outDir1 = '/host/percy/local_raid/hans/amyg/func/'


In [2]:
##retrieve U1 and U2 maps in func space to partition them into groups for the highest and lowest 25% values
#iterate through each subject

###threshold the top and bttom X% values of U1 and U2
thrshhld=25
gauss=10

#subjList=[1,4,5,6,7,9,10,11,12,13]
subjList=[1,4]

for i in subjList:
    if i<10:
        subject = str('PNC00'+str(i))
    else:
        subject = str('PNC0'+str(i))
    #swith U1 and U2 of PNC011 because of UMAP orientation
    if i == 11:
        mapNif1 = nib.load(workDir+subject+'_01_amyg_U2_funcSpace_1sd.nii.gz')
        mapU1 = np.array(mapNif1.dataobj)
        mapNif2 = nib.load(workDir+subject+'_01_amyg_U1_funcSpace_1sd.nii.gz')
        mapU2 = np.array(mapNif2.dataobj)
    else:
        mapNif1 = nib.load(workDir+subject+'_01_amyg_U1_funcSpace_1sd.nii.gz')
        mapU1 = np.array(mapNif1.dataobj)
        mapNif2 = nib.load(workDir+subject+'_01_amyg_U2_funcSpace_1sd.nii.gz')
        mapU2 = np.array(mapNif2.dataobj)
    #retreieve amygdala mask to be able to have array of u1 and u2 values only within amygdala borders
    maskNif = nib.load(workDir+subject+'_01_amyg_mask_funcSpace_1sd.nii.gz')
    mask = np.array(maskNif.dataobj)
    
    mask = np.where(mask == 1, 0, 1)
    maskBin = mask.tolist()
    #print(maskBin[1][1][:])
    U1vec = ma.masked_array(mapU1, maskBin)
    U1vec = U1vec.compressed()
    
    U2vec = ma.masked_array(mapU2, maskBin)
    U2vec = U2vec.compressed()
    
    #make array which contains coordinate values along with u1 and u2 values or each voxel
    #print(U1vec)
    print(len(U1vec))
    #print(U2vec)
    print(len(U2vec))    
    coords=np.zeros((12214,5))
    num=0
    for i in range(len(mapU1[:,1,1])):
        for j in range(len(mapU1[1,:,1])):
            for k in range(len(mapU1[1,1,:])):
                if mapU1[i,j,k] != 0:
                    coords[num,0]=i
                    coords[num,1]=j
                    coords[num,2]=k
                    coords[num,3]=mapU1[i,j,k]
                    coords[num,4]=mapU2[i,j,k]
                    num=num+1
    print(num)
    num=0
    for i in range(len(mapU2[:,1,1])):
        for j in range(len(mapU2[1,:,1])):
            for k in range(len(mapU2[1,1,:])):
                if mapU1[i,j,k] != 0:
                    coords[num,0]=i
                    coords[num,1]=j
                    coords[num,2]=k
                    coords[num,3]=mapU1[i,j,k]
                    coords[num,4]=mapU2[i,j,k]
                    num=num+1
    print(num)
    
    ###threshold the top and bttom 25% values of U1 and U2
    #U1
    percentage=0.01*thrshhld
    threshold=int(percentage*len(U1vec))
    #print(threshold)
    
    #depending on whether subject have positive or negative correlations with U1 and the two main axes deifned by U1(medial lateral and inferior superior) we will inverse the top of bottom values to stay consistent
    #subjects that have negative correlations:PNC001,PNC007,PNC009,PNC012  #make code more generalizable by choosing which ones to inverse by looking directly at the correlation values
    if subject == 'PNC001' or subject == 'PNC007' or subject == 'PNC009' or subject == 'PNC012':
        u1bot=np.argpartition(U1vec,-threshold)[-threshold:]
        u1top=np.argpartition(U1vec,threshold)[:threshold]
        print('inverted')
    else:
        u1top=np.argpartition(U1vec,-threshold)[-threshold:]
        u1bot=np.argpartition(U1vec,threshold)[:threshold]
        print('not inverted')
    u2top=np.argpartition(U2vec,-threshold)[-threshold:]
    u2bot=np.argpartition(U2vec,threshold)[:threshold]
    path=str(workDir+subject+'_01_amyg_func_space-func_desc-me_timeseries_clean.txt')
    tseries=np.loadtxt(path,delimiter=' ')
    print(tseries.shape)
    #get the average values of all timeseries for each partition (u1top25 U1bot25...)
    weights=np.zeros(len(U1vec))
    for i in u1top:
        weights[i]=1
    print(weights)
    u1topavg=np.average(tseries,axis=1,weights=weights)

    weights=np.zeros(len(U1vec))
    for i in u1bot:
        weights[i]=1
    #print(weights)
    u1botavg=np.average(tseries,axis=1,weights=weights)
    #print(topavg.shape)

    #U2
    weights=np.zeros(len(U1vec))
    for i in u2top:
        weights[i]=1
    #print(weights)
    u2topavg=np.average(tseries,axis=1,weights=weights)

    weights=np.zeros(len(U1vec))
    for i in u2bot:
        weights[i]=1
    #print(weights)
    u2botavg=np.average(tseries,axis=1,weights=weights)
    #print(topavg.shape)

    #retrieve average value for whole amygdala
    for i in mask:
        weights[i]=1
    amyg_timeseries=np.average(tseries,axis=1,weights=weights)
    
    ###retrieve the cortex timeseries for the subject
    funcDir=str('/data_/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-'+subject+'/ses-01/func/desc-me_task-rest_bold/')
    path=str(funcDir+'surf/sub-'+subject+'_ses-01_surf-fsLR-32k_desc-timeseries_clean.shape.gii')
    tmp=nib.load(path)
    cortexTseries=np.array([x.data for x in tmp.darrays])
    cortexTseries=cortexTseries[0]
    #cortexTseries=np.loadtxt(path)
    
    print('this is the shape of the timeseries')
    print(cortexTseries.shape)
    ###gaussian smooth for the timeseries values
    surf_lh, surf_rh = load_conte69()
    lh_smooth=mesh.mesh_smooth(cortexTseries[:,:32492],surf_lh,gauss)
    rh_smooth=mesh.mesh_smooth(cortexTseries[:,32492:],surf_rh,gauss)
    cortexTseries = np.append(lh_smooth,rh_smooth,axis=1)
    
    
    ###get pearson correlation of the avg of top and bottom 25% of u1 nd u2 values with the cortex timeseries
    from scipy import stats
    Pcorr=np.zeros((4,len(cortexTseries[1,:])))
    tmp=np.zeros(len(cortexTseries[1,:]))
    #also get the pearson correlation of the avg amygdala values with the cortex timeseries
    PcorrAmyg=np.zeros((1,len(cortexTseries[1,:])))
    for i in range(len(cortexTseries[1,:])):
        res1=stats.pearsonr(u1topavg,cortexTseries[:,i])
        res2=stats.pearsonr(u1botavg,cortexTseries[:,i])
        res3=stats.pearsonr(u2topavg,cortexTseries[:,i])
        res4=stats.pearsonr(u2botavg,cortexTseries[:,i])
        Pcorr[0,i]=res1[0]
        Pcorr[1,i]=res2[0]
        Pcorr[2,i]=res3[0]
        Pcorr[3,i]=res4[0]
        
        #get the pearson correlation of the avg amygdala values with the cortex timeseries
        res5=stats.pearsonr(amyg_timeseries,cortexTseries[:,i])
        PcorrAmyg[0,i]=res5[0]
    
    
    
    ###save all the correlation values in a 4 X 64984 matrix where it is u1top25, u1bot25, u2top25, u2bot25
    print(len(cortexTseries[1,:]))
    print(Pcorr.shape)
    path=str(outDir1+subject+'_U1U2_corr_top_bot_'+str(thrshhld)+'_with_cortex_gs'+str(gauss)+'.txt')
    #np.savetxt(path,Pcorr)
    
    #also save correlation values of whole amyg
    path=str(outDir1+subject+'_whole_amyg_corr_with_cortex_gs'+str(gauss)+'.txt')
    #np.savetxt(path,PcorrAmyg)

    #save correlations with gaussian smooth for the timeseries values
    #path=str(outDir1+subject+'_U1U2_corr_top_bot_10_with_cortex_smoothed.txt')
    #np.savetxt(path,Pcorr)

138
138
138
138
inverted
(275, 138)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0.
 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1.]
this is the shape of the timeseries
(275, 64984)
 275 x 1 surfaces to smooth, % remaining: 100 
90 80 70 60 50 40 30 20 10 0 Done
 275 x 1 surfaces to smooth, % remaining: 100 
90 80 70 60 50 40 30 20 10 0 Done
64984
(4, 64984)
106
106
106
106
not inverted
(275, 106)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1

In [3]:
#z-score for each subject U1 and U2 partitions
import scipy.stats as stats
def fisher_z(r):
    tmp=np.log(1+r)-np.log(1-r)
    return 0.5*tmp


for i in subjList:
    if i<10:
        subject = str('PNC00'+str(i))
    else:
        subject = str('PNC0'+str(i))
    pathIn=str(outDir1+subject+'_U1U2_corr_top_bot_'+str(thrshhld)+'_with_cortex_gs'+str(gauss)+'.txt')
    Pcorr=np.loadtxt(pathIn)
    #Pcorr=np.arctanh(Pcorr)
    #Pcorr=np.nan_to_num(Pcorr)
    z_connectivity = map(fisher_z,Pcorr)
    Pcorr = np.array(list(z_connectivity))

    pathOut=str(outDir1+subject+'_U1U2_fisherZ_top_bot_'+str(thrshhld)+'_with_cortex_gs'+str(gauss)+'.txt')
    #np.savetxt(pathOut,Pcorr)
    
    #(Z-score) do the same but with the whole amygdala, z-score the correlations of whole amyg and cortex timeseries
    path1=str(outDir1+subject+'_whole_amyg_corr_with_cortex_gs'+str(gauss)+'.txt')
    PcorrAmyg=np.loadtxt(path1)
    #PcorrAmyg=np.arctanh(PcorrAmyg)
    z_connectivity=map(fisher_z,PcorrAmyg)
    PcorrAmyg = np.array(list(z_connectivity))

    path2=str(outDir1+subject+'_fisherZ_whole_amyg_with_cortex_gs'+str(gauss)+'.txt')
    #np.savetxt(path2,PcorrAmyg)

In [ ]:
#makes hemisphere plots of z-scored average pearson correlation of all subject timeseries FOR WHOLE AMYGDALA
from brainspace.plotting import plot_hemispheres
from brainspace.utils.parcellation import map_to_labels
from brainspace.datasets import load_group_fc, load_parcellation, load_conte69
import brainstat.mesh.data as mesh
from brainstat.datasets import fetch_mask, fetch_template_surface
from brainstat.tutorial.utils import fetch_mics_data
corr_all_subs=np.zeros((1,10,64984))
for count, i in enumerate([1,4,5,6,7,9,10,11,12,13]):
    if i<10:
        subject = str('PNC00'+str(i))
    else:
        subject = str('PNC0'+str(i))
    path=str(outDir1+subject+'_fisherZ_whole_amyg_with_cortex_gs'+str(gauss)+'.txt')
    PcorrAmyg=np.loadtxt(path)
    corr_all_subs[0][count][:]=PcorrAmyg

PcorrAmyg=np.average(corr_all_subs[0][:][:],axis=0)

surf_lh, surf_rh = load_conte69()

plot_hemispheres(surf_lh, surf_rh, PcorrAmyg, color_bar=True, color_range=(0, 0.15),
        label_text=['amygdala'], cmap="viridis", 
        embed_nb=True, size=(1400, 400), zoom=1.3, nan_color=(0.7, 0.7, 0.7, 1), 
        cb__labelTextProperty={"fontSize": 12}, interactive=False)


/export02/data/jessica/miniconda3/envs/python37/lib/python3.7/site-packages/nilearn/datasets/__init__.py:89: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [ ]:
#makes hemisphere plots of z-scored average pearson correlation of all subject timeseries FOR U1 AND U2 TOP AND BOTTOM 25%
from brainspace.plotting import plot_hemispheres
from brainspace.utils.parcellation import map_to_labels
from brainspace.datasets import load_group_fc, load_parcellation, load_conte69
import brainstat.mesh.data as mesh
from brainstat.datasets import fetch_mask, fetch_template_surface
from brainstat.tutorial.utils import fetch_mics_data
corr_all_subs=np.zeros((2,10,64984))
for count, i in enumerate(subjList):
    if i<10:
        subject = str('PNC00'+str(i))
    else:
        subject = str('PNC0'+str(i))
    path=str(outDir1+subject+'_U1U2_fisherZ_top_bot_'+str(thrshhld)+'_with_cortex_gs'+str(gauss)+'.txt')
    Pcorr=np.loadtxt(path)
    Pcorr=Pcorr[:2][:]

    corr_all_subs[0][count][:]=Pcorr[0][:]
    corr_all_subs[1][count][:]=Pcorr[1][:]
    #cor=map_to_labels(Pcorr)

Pcorr[0,:]=np.average(corr_all_subs[0][:][:],axis=0)
Pcorr[1,:]=np.average(corr_all_subs[1][:][:],axis=0)

surf_lh, surf_rh = load_conte69()

#get the Yeo parcellations in order to identify the medial wall and turn the values inside into None for plotting purposes
from nibabel import gifti
ParcelDir='/data/mica1/03_projects/jessica/micasoft/parcellations/fs_LR-conte69/maps/'
YeoLHPath=str(ParcelDir+'lh.Yeo2011_7Networks_N1000.label.gii')
YeoRHPath=str(ParcelDir+'rh.Yeo2011_7Networks_N1000.label.gii')
YeoLH=nib.load(YeoLHPath)
YeoRH=nib.load(YeoRHPath)
tmp = YeoLH.agg_data()
YeoLRH=np.append(tmp,YeoRH.agg_data())
medial_wall=np.where(YeoLRH==0)
print(medial_wall[0])
for i in range(len(medial_wall[0])):
    Pcorr[1][medial_wall[0][i]]=None
    Pcorr[0][medial_wall[0][i]]=None

    
#uncomment this code if want to only show correclation values above 0.2 and below -0.2
#for i in range(len(Pcorr[0][:])):
#    if Pcorr[0][i]>-0.1 and Pcorr[0][i]<0.1:
#        Pcorr[0][i]=None
#    if Pcorr[1][i]>-0.1 and Pcorr[1][i]<0.1:
#        Pcorr[1][i]=None

    #plot the hemispheres with correlation values betwen both regions of the amygdala and the cortex
plot_hemispheres(surf_lh, surf_rh, Pcorr[:2,:], color_bar=True, color_range=(-0.15, 0.15),
        label_text=['U1top25','U1bot25'], cmap="bwr", 
        embed_nb=True, size=(1200, 700), zoom=1.2, nan_color=(0.7, 0.7, 0.7, 1), 
        cb__labelTextProperty={"fontSize": 12}, interactive=False)


In [ ]:
#visual test to see the different probability maps of each subject in plot hemispheres for U1 and U2
snap_20=np.zeros((20,64984))
for i in range(10):
    snap_20[i*2]=corr_all_subs[0][i][:]
    snap_20[i*2+1]=corr_all_subs[1][i][:]

print(snap_20.shape)
print(snap_20[0])
#only showing 17 of 20 different datasets since plot_hemispheres does not work with more
plot_hemispheres(surf_lh, surf_rh, snap_20[:17,:], color_bar=True, color_range=(0, 0.25),
        label_text=['1U1top25','1U1bot25','4U1top25','4U1bot25','5U1top25','5U1bot25','6U1top25','6U1bot25','7U1top25','7U1bot25','9U1top25','9U1bot25','10U1top25','10U1bot25','11U1top25','11U1bot25','12U1top25'],#'U1bot25','U1top25','U1bot25'],
        cmap="viridis", embed_nb=True, size=(1000, 2800), zoom=1.2,cb__labelTextProperty={"fontSize": 12},interactive=False,nan_color=(0.7, 0.7, 0.7, 1))

In [ ]:
###make mixed effects model for the timeseries treat subjects as random variable and look at differences between segments (Only analysis on U1)
thrshhld=25
gauss=10
mmMatrix=np.zeros((40,64984))
counter=0
for i in subjList:
    if i<10:
        subject = str('PNC00'+str(i))
    else:
        subject = str('PNC0'+str(i))
    path=str(outDir1+subject+'_U1U2_fisherZ_top_bot_'+str(thrshhld)+'_with_cortex_gs'+str(gauss)+'.txt')
    #path=str(outDir1+subject+'_U1U2_fisherZ_top_bot_'+str(thrshhld)+'_with_cortex_test.txt')
    fisherZ=np.loadtxt(path)
    mmMatrix[counter,:]=fisherZ[0,:]
    mmMatrix[counter+9,:]=fisherZ[2,:]
    mmMatrix[counter+18,:]=fisherZ[1,:]
    mmMatrix[counter+27,:]=fisherZ[3,:]
    counter=counter+1
#isolate only U1 data since further analysis will only be with U1
mmMatrixU1=np.delete(mmMatrix,[10,11,12,13,14,15,16,17,18,19],axis=0)
mmMatrixU1=np.delete(mmMatrixU1,[-10,-9,-8,-7,-6,-5,-4,-3,-2,-1],axis=0)

In [ ]:
#make mixed effects model of different network connectivities between subregions while accounting for different subjects
from brainstat.stats.terms import MixedEffect, FixedEffect
from brainstat.stats.SLM import SLM
import pandas as pd
from brainstat.datasets import fetch_mask, fetch_template_surface
mask=fetch_mask('fslr32k')
surf_combined = fetch_template_surface('fslr32k', join=True)

subU1=np.array(['sub1','sub1','sub2','sub2','sub3','sub3','sub4','sub4','sub5','sub5','sub6','sub6','sub7','sub7','sub8','sub8','sub9','sub9','sub10','sub10'])
segU1=np.array(['upper','lower','upper','lower','upper','lower','upper','lower','upper','lower','upper','lower','upper','lower','upper','lower','upper','lower','upper','lower'])

attributes={'sub':subU1,'seg':segU1}


df=pd.DataFrame(data=mmMatrixU1)
term_sub = MixedEffect(attributes['sub'])

term_seg = FixedEffect(attributes['seg'])
contrast_seg=(segU1 == "upper").astype(int) - (segU1 == "lower").astype(int)
model_mixed = term_sub + term_seg

slm_mixed = SLM(
    model_mixed,
    contrast_seg,
    surf='fslr32k',
    mask=mask,
    correction=["rft"],
    cluster_threshold=0.01,
    two_tailed=False,
)

slm_mixed.fit(mmMatrixU1)
#skwn, krts = slm_mixed.qc(mmMatrixU1, v=87)

In [ ]:
#look at the t-values of differences in connectivity for all regions of cortex 
plot_hemispheres(surf_lh, surf_rh, slm_mixed.t, color_bar=True, color_range=(-4,4),
        label_text=["t-values"], cmap="bwr", embed_nb=True, size=(1200, 350), zoom=1.2,
        nan_color=(0.7, 0.7, 0.7, 1), cb__labelTextProperty={"fontSize": 12}, interactive=False)


In [ ]:
len(slm_mixed.P["pval"]["C"])

In [ ]:
cp = [np.copy(slm_mixed.P["pval"]["C"])]
[np.place(x, np.logical_or(x > 0.05, ~mask), np.nan) for x in cp]
sigT=[np.copy(slm_mixed.P["pval"]["C"])]
for i,val in enumerate(slm_mixed.P["pval"]["C"]):
    if val < 0.05:
        sigT[0][i]=slm_mixed.t[0][i]
    else:
        sigT[0][i]=np.nan

plot_hemispheres(surf_lh, surf_rh, sigT, color_bar=True, color_range=(3,5),
        label_text=["Cluster p-values"]#, "Peak p-values", "Vertex p-values"]
        , cmap="hot", embed_nb=True, size=(1200, 350), zoom=1.2, nan_color=(0.7, 0.7, 0.7, 1), 
        cb__labelTextProperty={"fontSize": 12}, interactive=False)

In [ ]:
#find which regions are significantly different
cp = [np.copy(slm_mixed.P["pval"]["C"])]
[np.place(x, np.logical_or(x > 0.05, ~mask), np.nan) for x in cp]

pp = [np.copy(slm_mixed.P["pval"]["P"])]
[np.place(x, np.logical_or(x > 0.05, ~mask), np.nan) for x in pp]

vals = np.vstack([cp[0].T, pp[0].T])

#qp = [np.copy(slm_mixed.Q)]
#[np.place(x, np.logical_or(x > 0.05, ~mask), np.nan) for x in qp]

#vals = np.vstack([cp[0].T, pp[0].T, qp[0].T])

plot_hemispheres(surf_lh, surf_rh, vals[0], color_bar=True, color_range=(0, 0.1),
        label_text=["Cluster p-values"]#, "Peak p-values", "Vertex p-values"]
        , cmap="binary", embed_nb=True, size=(1200, 350), zoom=1.2, nan_color=(0,0,0,1), 
        cb__labelTextProperty={"fontSize": 12}, interactive=False)


In [ ]:
print(slm_mixed.P["clus"])
print(slm_mixed.P["peak"][0])

In [ ]:
#show the hemisphere with significant peak and cluster pvalue areas
cp = [np.copy(slm_mixed.P["pval"]["C"])]
[np.place(x, np.logical_or(x > 0.05, ~mask), np.nan) for x in cp]

pp = [np.copy(slm_mixed.P["pval"]["P"])]
[np.place(x, np.logical_or(x > 0.05, ~mask), np.nan) for x in pp]

#qp = [np.copy(slm_mixed.Q)]
#[np.place(x, np.logical_or(x > 0.05, ~mask), np.nan) for x in qp]

vals = np.vstack([cp[0].T, pp[0].T])#, qp[0].T])

plot_hemispheres(surf_lh, surf_rh, vals, color_bar=True, color_range=(0, 0.05),
        label_text=["Cluster p-values", "Peak p-values"], cmap="autumn_r", 
        embed_nb=True, size=(1400, 400), zoom=1.8, nan_color=(0.7, 0.7, 0.7, 1), 
        cb__labelTextProperty={"fontSize": 12}, interactive=False)
